## Setup and configuration

1. Firstly, we'll import usefull packages.
2. Then, we'll load the data, before visualize and preprocess it.
3. We'll try a simple CNN model and then we will evaluate its performances.
4. And finally, we'll use techniques such as data augmentation, learning rate decay and dropout to increase our model's accuracy.

**Import Packages**

In [0]:
import numpy as np                          # linear algebra
import os                                   # used for loading the data
from sklearn.metrics import confusion_matrix# confusion matrix to carry out error analysis
import seaborn as sn                        # heatmap
from sklearn.utils import shuffle           # shuffle the data
import matplotlib.pyplot as plt             # 2D plotting library
import tensorflow as tf                     # best library ever

Kaggle API setup

In [0]:
!mkdir .kaggle

import json
token = {"username":"npqbuu","key":"224292a46181101043b9eaa22bc77a39"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [5]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json

- path is now set to: {/content}


In [0]:
# Download dataset from Kaggle

!kaggle datasets download -d moltean/fruits -p /content
!unzip \*.zip

**Import datas**

In [0]:
# Global varriables
train_data_dir = 'fruits-360/Training'
test_data_dir = 'fruits-360/Test'

img_height = 100
img_width = 100
batch_size = 32

In [8]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

#Load dataset
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical',
    subset='validation') # set as validation data

test_generator = train_datagen.flow_from_directory(
    test_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='categorical') # set as test data

Using TensorFlow backend.


Found 45852 images belonging to 114 classes.
Found 11424 images belonging to 114 classes.
Found 19548 images belonging to 114 classes.


**Let's explore the dataset**

We can, for example, ask ourselves:

- How many training and testing examples do we have ?
- What is the size of the images ?
- What is the proportion of each observed category ?

In [10]:
train_generator.image_shape

(100, 100, 3)

## Model evaluation
Steps are:
1. Build the model
2. Compile the model
3. Train / fit the data to the model
4. Evaluate the model on the testing set
5. Carry out an error analysis of our model.

We can build an easy model composed of different layers such as:
- Conv2D: (32 filters of size 3 by 3) The features will be "extracted" from the image.
- MaxPooling2D: The images get half sized.
- Flatten: Transforms the format of the images from a 2d-array to a 1d-array of 100 100 3 pixel values.
- Relu : given a value x, returns max(x, 0).
- Softmax: 6 neurons, probability that the image belongs to one of the classes.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (100, 100, 3)), # the nn will learn the good filter to use
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

print(model.summary())

Then, we can compile it with some parameters such as:
- Optimizer: adam = RMSProp + Momentum. What is Momentum and RMSProp ?
- Momentum = takes into account past gradient to have a better update.
- RMSProp = exponentially weighted average of the squares of past gradients.
- Loss function: we use sparse categorical crossentropy for classification, each images belongs to one class only

In [0]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

We fit the model to the data from the training set. The neural network will learn by itself the pattern in order to distinguish each category.

Fitting/Training the model

In [0]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n //validation_generator.batch_size

model.fit_generator(generator = train_generator,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_data = validation_generator,
                    validation_steps = STEP_SIZE_VALID,
                    epochs = 10
)

Save model

In [0]:
model.save('my_model.h5')

Evaluate the model

In [0]:
model = load_model('my_model.h5') # Load model

model.evaluate_generator(generator=validation_generator, steps=STEP_SIZE_VALID)

Predict the output

In [0]:
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)